***
## Записываем дни рождения в базу

В проекте **acme_project** настроим приложение **birthday** так, чтобы данные, отправленные через форму, не пропадали бесследно при закрытии браузера, а сохранялись в БД. Получится хороший сервис, который поможет вспомнить о днях рождения друзей и коллег.

Создадим в приложении **birthday** модель `Birthday`. Чтобы сохранить идею и смысл приложения — спишем поля модели с формы `BirthdayForm`.

```py
# birthday/models.py
from django.db import models


class Birthday(models.Model):
    first_name = models.CharField('Имя', max_length=20)
    last_name = models.CharField(
        'Фамилия', blank=True, help_text='Необязательное поле', max_length=20
    )
    birthday = models.DateField('Дата рождения') 
```
Разместите этот код в файле *birthday/models.py*.

Поля модели `Birthday` повторяют поля формы `BirthdayForm`:

```py
# birthday/forms.py
from django import forms


class BirthdayForm(forms.Form):
    first_name = forms.CharField(label='Имя', max_length=20)
    last_name = forms.CharField(
        label='Фамилия', required=False, help_text='Необязательное поле'
    )
    birthday = forms.DateField(
        label='Дата рождения',
        widget=forms.DateInput(attrs={'type': 'date'}),
    ) 
```



Есть и небольшие отличия: 

* в моделях вместо `required=False` применяется параметр `blank=True` (допустимы пустые значения);

* для полей модели с классом `models.CharField` обязательно должна быть указана максимальная длина поля; в нашем случае — `max_length=20`;

* кастомное название поля формы указывается в параметре `label`, а в модели это название указывают первым позиционным аргументом или в параметре `verbose_name`.

***
## Форма на основе модели: класс ModelForm

Модель готова, теперь можно создать форму, данные из которой будут передаваться в объект этой модели.

Перепишем класс `BirthdayForm`: сгенерируем форму, унаследовавшись от встроенного класса **ModelForm**: он создаёт формы на основе заданной модели. Класс `ModelForm` импортируется из модуля `forms`, оттуда же, откуда и класс `Form`.

При создании формы на основе модели нужно

1. Создать класс, унаследованный от `forms.ModelForm`.

2. В подклассе `Meta` созданного класса указать модель, на основе которой должна быть построена форма.

3. Указать поля, которые должны отображаться в форме. В форме необязательно использовать все поля модели. Определить список полей можно с помощью атрибутов `fields` или `exclude`:

* `fields = ('first_name', 'birthday')` — в форме будут показаны только перечисленные поля модели;

* `exclude = ('last_name',)` — в форме будут показаны все поля модели, за исключением перечисленных;

* если в форме нужно использовать все поля, то указывается значение `fields = '__all__'`.

    «Явное лучше неявного», так сказано в “The Zen of Python”.

    Перечень полей, отображаемых в форме, лучше указывать явным образом. В ином случае можно по недосмотру вывести в форму поля, значения которых должны устанавливаться автоматически, например внешний ключ (Foreign Key) объекта.

    Можно скрыть в форме и те поля, которые должны быть заполнены позже, а не сразу при создании объекта.

В файле *birthday/forms.py* измените класс `BirthdayForm`: 

* унаследуйте его от `forms.ModelForm`,

* удалите все существующие описания полей,

* в подклассе `Meta` укажите связанную с формой модель; укажите, что в форме должны использоваться все поля модели: `fields = '__all__'` — это наглядно продемонстрирует, к чему может привести забывчивость разработчика.

```py
# birthday/forms.py
from django import forms

# Импортируем класс модели Birthday.
from .models import Birthday


# Для использования формы с моделями меняем класс на forms.ModelForm.
class BirthdayForm(forms.ModelForm):
    # Удаляем все описания полей.

    # Все настройки задаём в подклассе Meta.
    class Meta:
        # Указываем модель, на основе которой должна строиться форма.
        model = Birthday
        # Указываем, что надо отобразить все поля.
        fields = '__all__' 
```

Чтобы форма работала как раньше — нужно указать, что для поля с датой рождения используется виджет с типом данных `date`. 

Виджеты полей описываются в подклассе `Meta`:

```py
class BirthdayForm(forms.ModelForm):

    class Meta:
        ...
        widgets = {
            'birthday': forms.DateInput(attrs={'type': 'date'})
        } 
```